## Import Library:

In [1]:
import os # untuk berinteraksi dengan sistem operasi, seperti untuk mengakses file dan direktori
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split # untuk membagi dataset menjadi subset training dan testing
import torch # Library utama PyTorch untuk komputasi tensor dan deep learning
from torch.utils.data import Dataset, DataLoader # Kelas untuk memanipulasi dataset dan membuat batch data untuk training
from torchvision import transforms # Modul untuk melakukan transformasi pada gambar seperti augmentasi data
import torch.nn as nn # Modul yang berisi berbagai komponen neural network seperti lapisan (layers), fungsi aktivasi, dll
import torch.optim as optim # Modul untuk algoritma optimasi, seperti SGD, Adam, dll
# from torchvision.models import resnet50 # Model pre-trained ResNet-50 yang bisa digunakan untuk tugas klasifikasi gambar
import torchvision.models as models
import time # Library untuk mengukur waktu eksekusi kode
from torch.autograd import Variable # Kelas yang membungkus tensor untuk melacak sejarah operasi dan menghitung gradient
from PIL import Image # Library untuk membuka, memanipulasi, dan menyimpan gambar
from sklearn.metrics import classification_report, confusion_matrix # Fungsi untuk menghitung dan menampilkan laporan klasifikasi dan confusion matrix
import cv2 # Library OpenCV untuk manipulasi gambar dan video
from torchvision.io import read_image

## Validation:

In [2]:
model = models.inception_v3(aux_logits = False, pretrained = False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/opt/conda/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [3]:
# Load State Dict dari File:
state_dict = torch.load('/kaggle/input/celeba-gender-classification-best-model/pytorch/facerecognition/3/model_best_Inception V3 Epoch 25 Weights BCE- Bella on Kaggle/model_best_Inception V3 Epoch 25 Weights BCE- Bella on Kaggle.pth')['model']

In [4]:
# Menghapus Kunci yang Tidak Sesuai dari State Dict (jika diperlukan):
keys_to_remove = ["AuxLogits.conv0.conv.weight", "AuxLogits.conv0.bn.weight", "AuxLogits.conv0.bn.bias", 
                  "AuxLogits.conv0.bn.running_mean", "AuxLogits.conv0.bn.running_var", "AuxLogits.conv0.bn.num_batches_tracked", 
                  "AuxLogits.conv1.conv.weight", "AuxLogits.conv1.bn.weight", "AuxLogits.conv1.bn.bias", 
                  "AuxLogits.conv1.bn.running_mean", "AuxLogits.conv1.bn.running_var", "AuxLogits.conv1.bn.num_batches_tracked", 
                  "AuxLogits.fc.weight", "AuxLogits.fc.bias"]
for key in keys_to_remove:
    if key in state_dict:
        del state_dict[key]

In [5]:
# Load State Dict ke Model:
model.load_state_dict(state_dict);

In [6]:
# Memindahkan Model ke GPU jika Tersedia:
if torch.cuda.is_available():
    model = model.to("cuda")

In [7]:
# Mendefinisikan Transformasi untuk Validasi:
from torchvision import transforms as T

val_transforms = T.Compose([
    T.Resize(299),
    T.CenterCrop(299),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [8]:
# Daftar gambar validasi
val_images_list = os.listdir('/kaggle/input/celeba-face-recognition/validation_images/validation_images')
val_images_list.sort()

In [9]:
val_images_list

['female_1.jpeg',
 'female_2.jpeg',
 'female_3.jpeg',
 'female_4.jpeg',
 'female_5.jpeg',
 'male_1.jpg',
 'male_2.jpeg',
 'male_3.jpeg',
 'male_4.jpeg',
 'male_5.jpeg']

In [10]:
# Prediksi Gambar Validasi:

inf_time_start = time.time()
val_pred = torch.empty((0)).cuda() if torch.cuda.is_available() else torch.empty((0))
for i in val_images_list:
    image_path = os.path.join('/kaggle/input/celeba-face-recognition/validation_images/validation_images', i)
    image = Image.open(image_path).convert('RGB')
    inputs = val_transforms(image)
    inputs = inputs.unsqueeze(0)
    if torch.cuda.is_available():
        inputs = inputs.cuda()

    with torch.no_grad():  # Nonaktifkan Gradient Calculation untuk Menghemat Memori
        outputs = model(inputs)
    
    preds = (torch.sigmoid(outputs).squeeze() > 0.5).float()
    if torch.cuda.is_available():
        val_pred = torch.cat((val_pred, preds.reshape(1).cuda()), -1)
    else:
        val_pred = torch.cat((val_pred, preds.reshape(1)), -1)

print('Prediction Time for Each Image is {} ms'.format(1000 * (time.time() - inf_time_start) / len(val_images_list)))

Prediction Time for Each Image is 135.53245067596436 ms


In [ ]:
# inf_time_start = time.time()
# val_pred = torch.empty((0)).cuda()
# for i in val_images_list:
#     image_path = os.path.join('/kaggle/input/celeba-face-recognition/validation_images/validation_images', i)
#     image = Image.open(image_path).convert('RGB')
#     inputs = val_transforms(image)
#     inputs = Variable(inputs.cuda())
#     inputs = inputs.unsqueeze(0)
#     outputs = model(inputs)
#     preds = (torch.sigmoid(outputs).squeeze() > 0.5).float()
#     val_pred = torch.cat((val_pred, preds.reshape(1)), -1)
# print('Prediction Time for Each Image is {} s'.format((time.time() - inf_time_start) / len(val_images_list)))

In [11]:
val_pred

tensor([1., 1., 1., 0., 0., 1., 0., 0., 1., 1.], device='cuda:0')